# Object Detection

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [35]:
from keras.layers import Input, Dense, Activation, BatchNormalization, Reshape, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.optimizers import Adam
from keras.models import Model
from keras.losses import categorical_crossentropy

In [49]:
def conv_block(block_input, filters, kernel_size, dropout_rate=0.4, pool_size=2, activation='relu'):
    x = Conv2D(filters=filters, kernel_size=kernel_size)(block_input)
#     x = BatchNormalization()(x)
    x = Activation(activation=activation)(x)
    x = Dropout(rate=dropout_rate)(x)
    return MaxPooling2D(pool_size=pool_size)(x)


def dense_block(block_input, layer_dim, dropout_rate, activation):
    x = Dense(units=layer_dim)(block_input)
    x = BatchNormalization()(x)
    x = Activation(activation=activation)(x)
    return Dropout(rate=dropout_rate)(x)


def create_model(model_shape, output_dim):
    nn_input = Input(shape=model_shape)
    
    x = conv_block(nn_input, filters=16, kernel_size=4, dropout_rate=0.3)
    x = conv_block(x, filters=32, kernel_size=3, dropout_rate=0.5)
    x = Flatten()(x)
    x = dense_block(x, layer_dim=50, dropout_rate=0.5, activation="relu")
    
    nn_pred = Dense(units=output_dim, activation="softmax")(x)
    
    model = Model(inputs=nn_input, outputs=nn_pred)
    
    model.compile(
        optimizer=Adam(),
        loss=categorical_crossentropy ,
        metrics=[ 'accuracy' ]
    )
    
    return model
    

In [60]:
model_parameters = {
    "model_shape": (32, 32, 1),
    "output_dim": 8
}
model = create_model(**model_parameters)

x_train = np.load("data/processed_data/train_x.npy")
y_train = np.load("data/processed_data/train_y.npy")
x_val = np.load("data/processed_data/test_x.npy")
y_val = np.load("data/processed_data/test_y.npy")

fit_params = {
    "x": x_train,
    "y": y_train,
    "epochs": 200,
    "batch_size": 32,
    "validation_data": (x_val, y_val) 
}

In [61]:
model.fit(**fit_params)

Train on 4139 samples, validate on 2760 samples
Epoch 1/200
4139/4139 [==============================] - 3s 713us/step - loss: 1.4904 - acc: 0.4849 - val_loss: 1.5032 - val_acc: 0.4576
Epoch 2/200
4139/4139 [==============================] - 2s 437us/step - loss: 1.0219 - acc: 0.6538 - val_loss: 1.3383 - val_acc: 0.5236
Epoch 3/200
4139/4139 [==============================] - 2s 517us/step - loss: 0.8619 - acc: 0.7002 - val_loss: 1.2703 - val_acc: 0.5391
Epoch 4/200
4139/4139 [==============================] - 2s 460us/step - loss: 0.7316 - acc: 0.7480 - val_loss: 1.3546 - val_acc: 0.4601
Epoch 5/200
4139/4139 [==============================] - 2s 487us/step - loss: 0.6805 - acc: 0.7560 - val_loss: 1.1213 - val_acc: 0.5667
Epoch 6/200
4139/4139 [==============================] - 2s 476us/step - loss: 0.6413 - acc: 0.7695 - val_loss: 1.1357 - val_acc: 0.5790
Epoch 7/200
4139/4139 [==============================] - 2s 441us/step - loss: 0.6039 - acc: 0.7913 - val_loss: 1.0625 - val_acc: 